In [24]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd


import os.path
CURDIR=os.path.abspath(".")
os.chdir("..")
from lib.maxcut_solver import maxcut_count,brute_search

In [25]:
def check_count(adj_m,conf,count,tolerance=0.02):
    return np.abs(maxcut_count(conf,adj_m)-count)<tolerance

def check_optimal(adj_m,count,tolerance=0.02):
    return np.abs(brute_search(adj_m)[-1]-count)<tolerance

In [51]:
def parse_data_file(file,method="benchmark",count_check=True,tolerance=0.02):
    '''
    parse data text file, convert string to data
    '''
    if count_check:
        adj_matrice=np.loadtxt("./data/random_adjancent_matrix.txt")
        N=adj_matrice.shape[0]
        n=int(np.sqrt(adj_matrice.shape[1]))
        
    
    def parse_benchmark(line,count_check=False,index=None):
        conf,count=line[1:-2].split(")")
        conf=tuple(map(lambda x: int((float(x)+1)/2),conf[7:-1].split(",")))
        count=float(count.split(",")[1])
        
        if count_check:
            adj_m=adj_matrice[index,:].reshape((n,n))
            count_check=check_count(adj_m,conf,count,tolerance)
            optimal_check=check_optimal(adj_m,count,tolerance)
            if count_check:
                return (conf,count,optimal_check)
            else:
                raise ValueError("incorrect edge count!")
        else:
            return (conf,count)
    
    def parse_solution(line,count_check=False,index=None):
        try:
            conf,count=line.strip().split("(")[-1].split("-")
            conf=tuple(map(int,conf[1:-3].split(",")))
            count=float(count[:-1])
            if count_check:
                adj_m=adj_matrice[index,:].reshape((n,n))
                count_check=check_count(adj_m,conf,count,tolerance)
                optimal_check=check_optimal(adj_m,count,tolerance)
                if count_check:
                    return (conf,count,optimal_check)
                else:
                    raise ValueError("incorrect edge count!")
                    return None
            else:
                return (conf,count) 
        except ValueError:
            return None
    
    parse_line=parse_solution
    
    if method=="benchmark":
        parse_line=parse_benchmark
    elif method=="solution":
        parse_line==parse_solution
    else:
        print("invalid input method")
        return np.nan
    
    with open(file) as f:
        lines=f.readlines()
        return [parse_line(lines[i],count_check,i) for i in range(len(lines))]

In [72]:
def result_benchmark(solution,benchmark,threshold=0.02):
    
    non_convergent=0
    worse_performance=0
    equal_performance=0
    better_performance=0
    
    
    for i in range(len(solution)):
        if solution[i] is None:
            non_convergent+=1
        else:
            
            if np.abs(solution[i][1]-benchmark[i][1])<threshold:
                    equal_performance+=1
            elif solution[i][1]-benchmark[i][1]<-threshold:
                worse_performance+=1
            elif solution[i][1]-benchmark[i][1]>threshold:
                better_performance+=1
                
    return [equal_performance,worse_performance,better_performance,non_convergent]

In [73]:
benchmark_file="./data/maxcut_solution.txt"

In [99]:
insight=[]
count_check=True
for i in range(1,5):
    solution_file="./data/maxcut_solution_n=5_p="+str(i)
    solution=parse_data_file(solution_file,method="solution",count_check=count_check)
    benchmark=parse_data_file(benchmark_file,method="benchmark",count_check=count_check)
    insight.append(result_benchmark(solution,benchmark))
    if count_check:
        insight[-1].append(sum([term[-1] for term in benchmark]))
        insight[-1].append(sum([term[-1] if term is not None else False for term in solution]))

In [100]:
insight=pd.DataFrame(data=insight,columns=["equal","worse","better","non_conv","gw_optimal","qaoa_optimal"])

In [101]:
insight.join(pd.Series(range(1,5),name="p_value")).set_index("p_value")#compare with goesman-williamson algorithm

,equal,worse,better,non_conv,gw_optimal,qaoa_optimal
p_value,,,,,,
1,103,21,35,41,138,120
2,98,28,42,32,138,128
3,106,26,39,29,138,132
4,113,8,47,32,138,156


In [18]:
os.chdir(CURDIR)